<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center"><a href="https://docs.vlm.run"><b>Website</b></a> | <a href="https://docs.vlm.run/"><b>API Docs</b></a> | <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a> | <a href="https://chat.vlm.run"><b>Chat</b></a>
</p>
</div>

# VLM Run Orion - Image-to-Image Style Transfer

This comprehensive cookbook demonstrates [VLM Run Orion's](https://vlm.run/orion) image-to-image style transfer capabilities. For more details on the API, see the [Agent API docs](https://docs.vlm.run/agents/introduction).

For this notebook, we'll cover how to use the **VLM Run Agent Chat Completions API** - an OpenAI-compatible interface for transferring artistic styles from one image to another with the same familiar chat-completions interface.

We'll cover the following topics:
 1. Basic Style Transfer (transfer style from one image to another)
 2. Artistic Style Transfer (painting styles, artistic movements)
 3. Photo-to-Painting Conversion
 4. Style Transfer with Content Preservation
 5. Multiple Style Variations
 6. Style Intensity Control
 7. Batch Style Transfer

## Prerequisites

- Python 3.10+
- VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))
- VLM Run Python Client with OpenAI extra `vlmrun[openai]`


## Setup

First, install the required packages and configure the environment.


In [ ]:
# Install required packages
%pip install vlmrun[openai] --upgrade --quiet
%pip install cachetools pillow requests numpy --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.3/151.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 16.5 MB/s eta 0:00:00


In [ ]:
import os
import getpass

VLMRUN_API_KEY = os.getenv("VLMRUN_API_KEY", None)
if VLMRUN_API_KEY is None:
    VLMRUN_API_KEY = getpass.getpass("Enter your VLM Run API key: ")


## Initialize the VLM Run Client

We use the OpenAI-compatible chat completions interface through the VLM Run SDK.


In [ ]:
from vlmrun.client import VLMRun

BASE_URL = os.getenv("VLMRUN_BASE_URL", "https://agent.vlm.run/v1")
client = VLMRun(api_key=VLMRUN_API_KEY, base_url=BASE_URL)
print("VLM Run client initialized successfully!")
print(f"Base URL: {BASE_URL}")


## Response Models (dtypes)

We define Pydantic models for structured outputs. These models include **cached properties** that automatically download and convert images from URLs to PIL Images for easy manipulation.


In [ ]:
from typing import List

from PIL import Image
from pydantic import BaseModel, Field
from vlmrun.common.utils import download_image
from vlmrun.types import ImageRef


class StyleTransferResponse(BaseModel):
    """Response containing the style-transferred image."""
    image: ImageRef = Field(..., description="The style-transferred image")


class MultipleStyleTransferResponse(BaseModel):
    """Response containing multiple style variations."""
    images: List[ImageRef] = Field(..., description="List of style-transferred images with different variations")

print("Response models defined successfully!")
print("Models include cached properties for automatic image downloading.")


## Helper Functions

We create helper functions to simplify making chat completion requests with structured outputs.


In [ ]:
import hashlib
import json
from typing import Any, Type, TypeVar

import cachetools
from IPython.display import HTML
from vlmrun.common.image import encode_image
from pydantic import BaseModel


T = TypeVar('T', bound=BaseModel)


def display(images: Image.Image | list[Image.Image], texts: list[str] | None = None, width: int = 300):
    if isinstance(images, Image.Image):
        images = [images]
    if texts is None:
        texts = [None] * len(images)
    elif isinstance(texts, str):
        texts = [texts]
    elif len(texts) != len(images):
        raise ValueError("`texts` must be a list of the same length as `images`")

    imgs_html = ""
    for image, text in zip(images, texts):
        W, H = image.size
        if W > width:
            H = int(H * width / W)
            W = width
            image = image.resize((W, H))
        im_bytes = encode_image(image, format="JPEG")
        imgs_html += f"<div style='display:inline-block; margin:5px; text-align:center'>"
        imgs_html += f"<img src='{im_bytes}' style='width:{width}px; border-radius:6px'>"
        if text:
            imgs_html += f"<div style='font-size:12px; color:#666; margin-top:5px'>{text}</div>"
        imgs_html += f"</div>"
    return HTML(f"<div style='display:flex; flex-wrap:wrap'>{imgs_html}</div>")


def custom_key(prompt: str, images: list[Image.Image] | list[str] | None = None, response_model: Type[T] | None = None, model: str = "vlmrun-orion-1:auto"):
    """Custom key for caching chat_completion."""
    image_keys = []
    for image in images:
        if isinstance(image, Image.Image):
            thumb = image.copy()
            thumb.thumbnail((128, 128))
            encoded = encode_image(thumb, format="JPEG")
            image_keys.append(encoded)
        elif isinstance(image, str):
            image_keys.append(image)

    response_key = hashlib.sha256(json.dumps(response_model.model_json_schema(), sort_keys=True).encode()).hexdigest() if response_model else ""
    return (prompt, tuple(image_keys), response_key, model)


@cachetools.cached(cache=cachetools.TTLCache(maxsize=1000, ttl=3600), key=custom_key)
def chat_completion(
    prompt: str,
    images: list[Image.Image] | list[str] | None = None,
    response_model: Type[T] | None = None,
    model: str = "vlmrun-orion-1:fast"
) -> tuple[str | BaseModel, str]:
    """
    Make a chat completion request with optional images and structured output.

    Args:
        prompt: The text prompt/instruction
        images: Optional list of images to process (either PIL Images or URLs)
        response_model: Optional Pydantic model for structured output
        model: Model to use (default: vlmrun-orion-1:fast)

    Returns:
        Parsed response model if response_model provided, else raw response text
    """
    content = []
    content.append({"type": "text", "text": prompt})

    if images:
        for image in images:
            if isinstance(image, str):
                assert image.startswith("http"), "Image URLs must start with http or https"
                content.append({"type": "image_url", "image_url": {"url": image, "detail": "auto"}})
            elif isinstance(image, Image.Image):
                content.append({"type": "image_url", "image_url": {"url": encode_image(image, format="JPEG"), "detail": "auto"}})
            else:
                raise ValueError("Images must be either PIL Images or URLs")

    kwargs = {
        "model": model,
        "messages": [{"role": "user", "content": content}]
    }

    if response_model:
        kwargs["response_format"] = {
            "type": "json_schema",
            "schema": response_model.model_json_schema()
        }

    response = client.agent.completions.create(**kwargs)
    response_text = response.choices[0].message.content

    if response_model:
        return response_model.model_validate_json(response_text), response.session_id

    return response_text, response.session_id

print("Helper functions defined!")


## Image-to-Image Style Transfer Capabilities

VLM Run agents can transfer artistic styles from one image to another, creating stunning visual transformations while preserving the content structure of the original image.


### 1. Artistic Style Transfer

Apply famous artistic styles (like Van Gogh, Monet, Picasso) to your images using style reference images.


In [ ]:
CONTENT_IMAGE_URL = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/agent-media/white-sports-car-isolated-white-vector.avif"
STYLE_IMAGE_URL = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/agent-media/Vincent-van-Gogh-The-Starry-Night.webp"

content_img = download_image(CONTENT_IMAGE_URL)
style_img = download_image(STYLE_IMAGE_URL)

print(">> ORIGINAL IMAGES")
display([content_img, style_img], texts=["Content Image", "Style Reference"], width=400)


In [ ]:
result, session_id = chat_completion(
    prompt="Transfer the artistic style from the second image to the first image. Preserve the content and structure of the first image while applying the visual style, colors, and brushstrokes from the second image.",
    images=[content_img, style_img],
    response_model=StyleTransferResponse,
    model="vlmrun-orion-1:auto"
)

print("\n>> RESPONSE")
print(result)

# Get the style-transferred image
styled_img: Image.Image = client.artifacts.get(session_id=session_id, object_id=result.image.id)
print("\n>> STYLE TRANSFERRED IMAGE")
display(images=[styled_img], texts=["Style Transferred Result"], width=600)

### 2. Photo-to-Painting Conversion

Convert photographs into paintings with various artistic styles.


In [ ]:
PHOTO_URL = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.agent/lunch-skyscraper.jpg"
WATERCOLOR_STYLE_URL = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/agent-media/background-art.jpg"

photo_img = download_image(PHOTO_URL)
watercolor_img = download_image(WATERCOLOR_STYLE_URL)

print(">> ORIGINAL IMAGES")
display([photo_img, watercolor_img], texts=["Original Photo", "Watercolor Style"], width=400)



In [ ]:
result, session_id = chat_completion(
    prompt="Convert the first image (a photograph) into a watercolor painting using the style from the second image. Apply the characteristic watercolor effects: soft edges, translucent layers, and flowing colors while preserving the main subjects and composition.",
    images=[PHOTO_URL, WATERCOLOR_STYLE_URL],
    response_model=StyleTransferResponse,
    model="vlmrun-orion-1:auto"
)

print("\n>> RESPONSE")
print(result)

painted_img: Image.Image = client.artifacts.get(session_id=session_id, object_id=result.image.id)
print("\n>> PAINTING RESULT")
display(images=[painted_img], texts=["Watercolor Painting"], width=600)

### 3. Style Transfer with Content Preservation

Transfer style while explicitly preserving important content elements like faces, objects, or specific details.


In [ ]:
# Example: Style transfer with face preservation
CONTENT_IMAGE_URL = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/agent_use_cases/Headshot/headshot.jpg"
ARTISTIC_STYLE_URL = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/agent-media/background-art.jpg"

content_img = download_image(CONTENT_IMAGE_URL)
style_img = download_image(ARTISTIC_STYLE_URL)

print(">> ORIGINAL IMAGES")
display([content_img, style_img], texts=["Content Image", "Artistic Style"], width=400)


In [ ]:
result, session_id = chat_completion(
    prompt="Apply the artistic style from the second image to the first image, but preserve the faces and important details of the people. Keep the faces recognizable while applying the style to the background and other elements.",
    images=[CONTENT_IMAGE_URL, ARTISTIC_STYLE_URL],
    response_model=StyleTransferResponse,
    model="vlmrun-orion-1:auto"
)

print("\n>> RESPONSE")
print(result)

styled_img: Image.Image = client.artifacts.get(session_id=session_id, object_id=result.image.id)
print("\n>> STYLE TRANSFERRED IMAGE")
display(images=[styled_img], texts=["Style Transfer with Face Preservation"], width=600)

### 4. Multiple Style Variations

Generate multiple style transfer variations in a single request to explore different artistic interpretations.


In [ ]:
# Example: Generate multiple style variations
CONTENT_IMAGE_URL = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.caption/car.jpg"
STYLE_IMAGE_URL = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/agent-media/background-art.jpg"

content_img = download_image(CONTENT_IMAGE_URL)
style_img = download_image(STYLE_IMAGE_URL)

print(">> ORIGINAL IMAGES")
display([content_img, style_img], texts=["Content Image", "Style Reference"], width=400)



In [ ]:
result, session_id = chat_completion(
    prompt="Transfer the style from the second image to the first image. Generate 3 different variations with varying intensity levels: one subtle, one moderate, and one strong style application. Each should preserve the content structure but apply the style differently.",
    images=[CONTENT_IMAGE_URL, STYLE_IMAGE_URL],
    response_model=MultipleStyleTransferResponse,
    model="vlmrun-orion-1:auto"
)

print("\n>> RESPONSE")
print(result)

# Get all style variations
styled_images: List[Image.Image] = [
    client.artifacts.get(session_id=session_id, object_id=img.id)
    for img in result.images
]

print("\n>> STYLE VARIATIONS")
display(
    styled_images,
    texts=[f"Variation {i+1}" for i in range(len(styled_images))],
    width=300
)

### 5. Batch Style Transfer

Apply the same style to multiple content images efficiently.


In [ ]:
# Example: Apply same style to multiple images
STYLE_IMAGE_URL = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/agent-media/background-art.jpg"
CONTENT_IMAGES = [
    "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.caption/car.jpg",
    "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.agent/lunch-skyscraper.jpg"
]

style_img = download_image(STYLE_IMAGE_URL)
content_imgs = [download_image(url) for url in CONTENT_IMAGES]

print(">> STYLE REFERENCE")
display([style_img], texts=["Style Reference"], width=400)



In [ ]:
print("\n>> ORIGINAL CONTENT IMAGES")
display(content_imgs, texts=[f"Content {i+1}" for i in range(len(content_imgs))], width=300)

In [ ]:
# Apply style to each content image
styled_results = []
for i, content_url in enumerate(CONTENT_IMAGES):
    result, session_id = chat_completion(
        prompt="Transfer the artistic style from the second image to the first image. Preserve the content structure while applying the visual style.",
        images=[content_url, STYLE_IMAGE_URL],
        response_model=StyleTransferResponse,
        model="vlmrun-orion-1:auto"
    )
    styled_img = client.artifacts.get(session_id=session_id, object_id=result.image.id)
    styled_results.append(styled_img)

print("\n>> STYLE TRANSFERRED RESULTS")
display(styled_results, texts=[f"Styled {i+1}" for i in range(len(styled_results))], width=300)

### 6. Advanced: Combining Multiple Styles

Combine elements from multiple style images to create unique artistic effects.


In [ ]:
# Example: Combine multiple styles
CONTENT_IMAGE_URL = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.caption/car.jpg"
STYLE_1_URL = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/agent-media/background-art.jpg"
STYLE_2_URL = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/agent-media/background_frame.jpg"

content_img = download_image(CONTENT_IMAGE_URL)
style1_img = download_image(STYLE_1_URL)
style2_img = download_image(STYLE_2_URL)

print(">> ORIGINAL IMAGES")
display([content_img, style1_img, style2_img], texts=["Content", "Style 1", "Style 2"], width=300)


In [ ]:
result, session_id = chat_completion(
    prompt="Transfer a combination of styles from the second and third images to the first image. Blend the color palette from the second image with the frame from the third image, creating a unique artistic fusion while preserving the content structure.",
    images=[CONTENT_IMAGE_URL, STYLE_1_URL, STYLE_2_URL],
    response_model=StyleTransferResponse,
    model="vlmrun-orion-1:auto"
)

print("\n>> RESPONSE")
print(result)

styled_img: Image.Image = client.artifacts.get(session_id=session_id, object_id=result.image.id)
print("\n>> COMBINED STYLE RESULT")
display(images=[styled_img], texts=["Combined Style Transfer"], width=600)


---

## Conclusion

This cookbook demonstrated the comprehensive style transfer capabilities of the **VLM Run Orion Image Agent API**.

### Key Takeaways

1. **OpenAI-Compatible Interface**: The API follows the OpenAI chat completions format, making it easy to integrate style transfer into existing workflows.
2. **Structured Outputs**: Use Pydantic models with `response_model` parameter to get type-safe, validated responses with automatic image retrieval.
3. **Flexible Prompting**: Natural language prompts allow you to control style intensity, preserve content, and combine multiple styles in a single request.
4. **Content Preservation**: You can explicitly instruct the model to preserve important elements like faces, objects, or specific details while applying style.
5. **Batch Processing**: Apply the same style to multiple images efficiently using loops and the caching mechanism.
6. **Multiple Variations**: Generate multiple style variations in a single request to explore different artistic interpretations.

### Best Practices

- **Style Intensity**: Use descriptive language ("subtle", "moderate", "strong") to control how much the style affects the content.
- **Content Preservation**: Be explicit about what elements to preserve (faces, objects, composition) when needed.
- **Model Selection**: Use `vlmrun-orion-1:auto` for complex style transfers and `vlmrun-orion-1:fast` for simpler tasks.
- **Caching**: The built-in caching mechanism helps reduce API calls when experimenting with the same images and prompts.

### Next Steps

- Explore the [VLM Run Documentation](https://docs.vlm.run) for more details
- Join our [Discord community](https://discord.gg/AMApC2UzVY) for support
- Check out more examples in the [VLM Run Cookbook](https://github.com/vlm-run/vlmrun-cookbook)
- Experiment with different style combinations and intensity levels

Happy creating! 🎨
